In [7]:
import json
import torch
from utils import *

In [8]:
with open('./output/pagesLines.json', 'r') as f:
  pages = json.load(f)
with open('./output/units.json', 'r') as f:
  units = json.load(f)

In [9]:
class Node:
  def __init__(self, data):
    self.siblings = None
    self.child = None
    self.data = data

  def compare_to(self, other):
    sxnv = np.array(self.data['normalizedVertices'])[:, 0]
    oxnv = np.array(other.data['normalizedVertices'])[:, 0]
    
    diff = sxnv.min() - oxnv.min()
    if np.abs(diff) < 0.04:
      return 0
    
    if diff > 0:
      return 1
    else:
      return -1

class Root:
  def __init__(self, data):
    self.child = None
    self.data = data

def insert(root, node):
  if root.data.compare_to(node.data) == 0: #same lavel; small different
    root.siblings = node
  else: #root.data < node.data
    root.child = node
    

In [10]:
def skip(text):
  text = text.strip()
  if text.startswith('รายละเอียดงบประมาณจำแนกตามงบรายจ่าย') \
    or text.startswith('เงินนอกงบประมาณ'):
    return True
  return False

In [23]:
entries = []
for unit in units:
  for page_idx in range(unit['start'], unit['end']):
    page = pages[page_idx]

    if not page['interested']: continue

    for line in page['lines']:
      # if it should be skipped
      if skip(line['text']): continue

      # first block is bullets
      block_text = line['blocks'][0]['text'].strip()

      if block_text == '(':
        for block in line['blocks'][1:]:
          t = block['text'].strip()
          block_text += t
          if t == ')': break

      bullet = get_patern_of_bullet(block_text)

      if bullet != 0 or block_text == 'ปี':
        entries.append([line])
      else:
        entries[-1].append(line)

In [24]:
for ent in entries:
  print('\t--', end=' ')
  for line in ent:
    print(''.join([block['text'] for block in line['blocks']]), end='')
  print()

	-- 8 รายการบุคลากร215,427,400 บาท ภาครัฐ 
	-- 1. งบบุคลากร 39,523,600 บาท 
	-- 1.1 เงินเดือนและค่าจ้างประจำ 33,834,500 บาท 
	-- 1.1.1 เงินเดือน 30,387,500 บาท 
	-- 1.1.2 ค่าจ้างประจำ 3,447,000 บาท 
	-- 1.2 ค่าจางชวคราว 2,256,700 บาท 
	-- 1.3 ค่าตอบแทนพนักงานราชการ 3,432,400 บาท 
	-- 2. งบดำเนินงาน บาท 8,722,100 
	-- 2.1 ค่าตอบแทน ใช้สอยและวัสดุ 8,722,100 บาท 
	-- (1) ค่าตอบแทนพิเศษข้าราชการที่ได้รับเงินเดือนเต็มขั้น 22,100 บาท 
	-- (2) ค่าเช่าบ้าน 1,212,000 บาท 
	-- (3) เงินประจำตำแหน่งผู้บริหารมีวาระ 3,381,600 บาท 
	-- (4) ค่าตอบแทนรายเดือนตำแหน่งประเภทผู้บริหารที่มีวาระ 3,381,600 บาท 
	-- (5) เงินประจําตำแหน่งผู้บริหารไม่มีวาระ 254,400 บาท 
	-- (6) ค่าตอบแทนรายเดือนตำแหน่งประเภทผู้บริหารที่ไม่มีวาระ 254,400 บาท 
	-- (7) เงินสมทบกองทุนประกันสังคม 207,000 บาท 
	-- (8) เงินสบทบกองทุนเงินทดแทน 9,000 บาท 
	-- 3. งบเงินอุดหนุน 167,181,700 บาท 
	-- 3.1 เงินอุดหนุนทั่วไป 167,181,700 บาท 
	-- 1) ค่าจ้างพนักงานมหาวิทยาลัย 167,181,700 บาท 
	-- 10 รายละเอียดงบประมาณจำแนกตามงบรายจ่าย ผลผลิต : ผู